In [2]:
from DataReader import DataReaderChallangePolimi2017
import implicit
from CB_asy import CB_asymmetric_cosine
from CF_IB_asy import CF_IB_asymmetric_cosine
from CF_UB_asy import CF_UB_asymmetric_cosine
from implicit.nearest_neighbours import all_pairs_knn
from sklearn.preprocessing import normalize
from sklearn.utils.sparsefuncs import (inplace_row_scale,inplace_column_scale)
import numpy as np
import scipy.sparse as sp
import pickle

evaluation = True
rebuild = False
dr = DataReaderChallangePolimi2017(evaluation=evaluation,rebuild=rebuild)

urm = dr.getURM_csr()
t_p = dr.target_playlists
t_t = dr.target_tracks    

Date Reader Challange Polimi 2017
Loading data from ./DumpData/DataReaderTrainTestDump
Dataset train-test loaded


In [42]:
class CF_IB_asymmetric_cosine:
    def __init__(self, urm, k=100, alpha=0.5, q=1, m=0):
        urm = sps.csr_matrix(urm, dtype=np.float32)
        #urm = dr.bm25_col(urm)
        if m == 0:
            m = 1e-6

        self.urm = urm
        self.k = k
        self.alpha = alpha
        self.q = q
        self.m = m
        self.MAP = 0.0

        n_user = urm.shape[0]
        n_item = urm.shape[1]
        self.n_playlists = n_user
        self.n_tracks = n_item

        print("Start asymmetric cosine item-based model..")
        s = (urm.T * urm).tocsr()
        s_diagonal = s.diagonal()
        s = s - sps.dia_matrix((s.diagonal()[np.newaxis, :], [0]), shape=s.shape)
        s.eliminate_zeros()
        s_one = sp.csr_matrix((np.ones(s.data.shape[0]), s.indices, s.indptr), shape=(n_item, n_item),
                               dtype=np.float32)
        norm1 = s_diagonal ** (1 - alpha)
        norm2 = s_diagonal ** alpha
        inplace_row_scale(s_one, norm1)
        inplace_column_scale(s_one, norm2)
        s_one.data = (s_one.data + m)
        s.data = s.data / s_one.data
        assert sps.isspmatrix_csr(s)
        data, rows, cols = [], [], []
        for i_row in range(s.shape[0]):
            if i_row%25000==0:
                print i_row
            row = s[i_row, :].toarray().ravel()
            topk_idxs_values = np.argpartition(row, -(k))[-(k):]
            #topk_idxs_values = np.argsort(row)[-(k):]
            n = topk_idxs_values.shape[0]
            # create incrementally the similarity matrix
            data.extend(row[topk_idxs_values])
            cols.extend(topk_idxs_values)
            rows.extend(np.full(n, i_row))
        s = sp.coo_matrix((data, (rows, cols)), shape=(s.shape[0], s.shape[0]), dtype=np.float32)
        s.eliminate_zeros()
        self.s = s.tocsr()
        return

In [43]:
m = CF_IB_asymmetric_cosine(urm,alpha=0.3)

Start asymmetric cosine item-based model..
0
25000
50000
75000


In [45]:
s = m.s
r = urm* s
#r = normalize(r,axis=0)
dr.evaluateMAP(r)

User 0 of 10000
User 2500 of 10000
User 5000 of 10000
User 7500 of 10000
Recommender performance is: MAP = 0.0701


0.070104722222222721